In [ ]:
!which python
!pip list | grep tensorflow

In [1]:
import os
import tensorflow as tf

# Caminho do modelo
tflite_path = "mad_model_multisensors.tflite"

# Remove o modelo anterior, se existir
if os.path.exists(tflite_path):
    os.remove(tflite_path)
    print("Modelo antigo removido.")

# Classe do modelo com múltiplos sensores
class MultiSensorMADModel(tf.Module):
    @tf.function(input_signature=[tf.TensorSpec([None, 4, None], tf.int32)])
    def __call__(self, inputs):
        def compute_mad(sensor_data):
            timestamps = sensor_data[0]
            x = tf.cast(sensor_data[1], tf.float32)
            y = tf.cast(sensor_data[2], tf.float32)
            z = tf.cast(sensor_data[3], tf.float32)

            mag = tf.sqrt(x*x + y*y + z*z)
            win = tf.cast(tf.floor(tf.cast(timestamps, tf.float32) / 5000.0), tf.int32)
            win0 = win - tf.reduce_min(win)
            num_wins = tf.reduce_max(win0) + 1

            sums = tf.math.unsorted_segment_sum(mag, win0, num_wins)
            counts = tf.math.unsorted_segment_sum(tf.ones_like(mag), win0, num_wins)
            means = sums / counts
            valid = counts > 0
            num_valid = tf.reduce_sum(tf.cast(valid, tf.float32))
            sum_means = tf.reduce_sum(tf.where(valid, means, 0.0))
            sum_sq = tf.reduce_sum(tf.where(valid, means * means, 0.0))
            mean_of_means = sum_means / num_valid
            mean_sq = sum_sq / num_valid
            std_dev = tf.sqrt(mean_sq - mean_of_means * mean_of_means)
            safe_min = tf.where(valid, means, 1e30)
            safe_max = tf.where(valid, means, -1e30)
            min_val = tf.reduce_min(safe_min)
            max_val = tf.reduce_max(safe_max)
            return tf.stack([mean_of_means, std_dev, min_val, max_val])

        results = tf.map_fn(compute_mad, inputs, fn_output_signature=tf.float32)
        return results

# Cria e exporta o modelo
mad_model = MultiSensorMADModel()
concrete_fn = mad_model.__call__.get_concrete_function(
    tf.TensorSpec(shape=[None, 4, None], dtype=tf.int32)
)

# Converte para TFLite
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_fn], mad_model)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS  # necessário por usar tf.map_fn e outros
]
tflite_model = converter.convert()

# Salva o modelo
with open(tflite_path, "wb") as f:
    f.write(tflite_model)
    print("Novo modelo salvo com sucesso.")

# Verificação
interpreter = tf.lite.Interpreter(model_path=tflite_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()

# ✅ Esperado: shape [num_sensores, 4, N]
print("Shape de entrada:", input_details[0]['shape'])
print("Shape signature :", input_details[0]['shape_signature'])
print("Tipo de entrada :", input_details[0]['dtype'])

ModuleNotFoundError: No module named 'tensorflow'

In [4]:
import numpy as np
import tensorflow as tf
import time

# Função de referência (Python puro)
def get_mad_py(data):
    timestamps = data[0]
    magnitudes = np.sqrt(np.sum(np.square(data[1:]), axis=0))
    magnitude_by_time = list(zip(timestamps, magnitudes))

    window_size_ms = 5000
    start_time = magnitude_by_time[0][0]
    end_time = magnitude_by_time[-1][0]

    block_avgs = []
    current_start = start_time

    while current_start <= end_time:
        current_end = current_start + window_size_ms
        values = [mag for ts, mag in magnitude_by_time if current_start <= ts < current_end]
        if values:
            block_avgs.append(np.mean(values))
        current_start = current_end

    mean = np.mean(block_avgs)
    std_dev = np.std(block_avgs)
    min_val = np.min(block_avgs)
    max_val = np.max(block_avgs)

    return np.array([mean, std_dev, min_val, max_val], dtype=np.float32)

# Gera dados simulados para múltiplos sensores
num_sensors = 32
N = 10000
timestamps = np.arange(N, dtype=np.int32) * 20

multi_sensor_data = []
for _ in range(num_sensors):
    x = np.random.randint(0, 10, N, dtype=np.int32)
    y = np.random.randint(0, 10, N, dtype=np.int32)
    z = np.random.randint(0, 10, N, dtype=np.int32)
    sensor_data = np.stack([timestamps, x, y, z], axis=0)  # shape [4, N]
    multi_sensor_data.append(sensor_data)

multi_sensor_input = np.stack(multi_sensor_data, axis=0)  # shape [num_sensors, 4, N]

# --- Tempo: Função Python puro para todos os sensores ---
start_cpu = time.time()
mad_refs = [get_mad_py(sensor) for sensor in multi_sensor_data]
mad_refs = np.stack(mad_refs)
end_cpu = time.time()
cpu_time_ms = (end_cpu - start_cpu) * 1000

# --- Tempo: Carregamento do modelo TFLite ---
start_load = time.time()
interpreter = tf.lite.Interpreter(model_path="mad_model_multisensors.tflite")
end_load = time.time()
load_time_ms = (end_load - start_load) * 1000

# Ajusta a entrada
interpreter.resize_tensor_input(0, [num_sensors, 4, N])
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# --- Tempo: Inferência TFLite ---
start_tflite = time.time()
interpreter.set_tensor(input_details[0]['index'], multi_sensor_input)
interpreter.invoke()
mad_out = interpreter.get_tensor(output_details[0]['index'])  # shape [num_sensors, 4]
end_tflite = time.time()
tflite_inference_time_ms = (end_tflite - start_tflite) * 1000

# Comparação sensor a sensor
print("📊 Comparação por sensor:")
for i in range(num_sensors):
    print(f"\n🔹 Sensor {i+1}")
    print("Referência (Python): ", mad_refs[i])
    print("Modelo TFLite      : ", mad_out[i])
    print("Erro absoluto      : ", np.abs(mad_refs[i] - mad_out[i]))

# Estatísticas gerais
reduction_pct = ((cpu_time_ms - tflite_inference_time_ms) / cpu_time_ms) * 100
print(f"\n⏱️ Tempo CPU total (Python):        {cpu_time_ms:.3f} ms")
print(f"📦 Tempo de carga TFLite:           {load_time_ms:.3f} ms")
print(f"⚡ Tempo de inferência TFLite:      {tflite_inference_time_ms:.3f} ms")
print(f"📉 Redução de tempo (inferência):  {reduction_pct:.2f}%")

ModuleNotFoundError: No module named 'numpy'